In [ ]:
### подключение библиотек

In [9]:
import pandas as pd

In [11]:
import numpy as np

In [13]:
import psycopg2

In [15]:
pd.__version__

'2.2.2'

In [17]:
np.__version__

'1.26.4'

In [19]:
psycopg2.__version__

'2.9.9 (dt dec pq3 ext lo64)'

### Подключение к БД"demo" используя запрос из аттестации

In [22]:
#запрос из аттестации
query = """
WITH all_tabl AS
(
SELECT 
air.aircraft_code,
air.model,
pass.seat_no,
se.fare_conditions,
COUNT(pass.seat_no) AS count_seat
FROM bookings.aircrafts AS air
Left Join bookings.seats AS se
ON air.aircraft_code = se.aircraft_code
Left Join bookings.boarding_passes AS pass
ON se.seat_no = pass.seat_no
GROUP BY
air.aircraft_code,
air.model,
se.fare_conditions,
pass.seat_no
ORDER BY
count_seat DESC
)
SELECT
all_tabl.aircraft_code,
all_tabl.model,
all_tabl.fare_conditions,
all_tabl.seat_no,
all_tabl.count_seat,
AVG(all_tabl.count_seat) OVER (PARTITION BY all_tabl.model) AS average ,
CASE WHEN (all_tabl.count_seat) >= (AVG(all_tabl.count_seat) OVER (PARTITION BY all_tabl.model)) THEN 'TOP' ELSE 'BASE' END status
FROM
all_tabl
ORDER BY
status DESC,
all_tabl.count_seat DESC,
average DESC;
"""

In [24]:
#подключение к БД "demo"
psy_df = psycopg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='Qwerty15'")

In [26]:
#запрос БД
df = pd.read_sql(query, con=psy_df)
df

C:\Users\user\AppData\Local\Temp\ipykernel_4296\444723592.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=psy_df)


,aircraft_code,model,fare_conditions,seat_no,count_seat,average,status
0,CN1,Сессна 208 Караван,Economy,1A,19462,14180.000000,TOP
1,CR2,Бомбардье CRJ-200,Economy,1A,19462,13973.160000,TOP
2,SU9,Сухой Суперджет-100,Business,1A,19462,12704.195876,TOP
3,319,Аэробус A319-100,Business,1A,19462,11671.629310,TOP
4,733,Боинг 737-300,Business,1A,19462,11160.692308,TOP
...,...,...,...,...,...,...,...
1334,773,Боинг 777-300,Economy,50K,635,3552.828358,BASE
1335,773,Боинг 777-300,Economy,33C,634,3552.828358,BASE
1336,773,Боинг 777-300,Economy,20J,632,3552.828358,BASE
1337,773,Боинг 777-300,Economy,35J,631,3552.828358,BASE


### количество строк / столбцов таблицы

In [46]:
df.shape

(1339, 7)

### тип данных в столбцах

In [30]:
df.dtypes

aircraft_code       object
model               object
fare_conditions     object
seat_no             object
count_seat           int64
average            float64
status              object
dtype: object

### количество уникальных значений по строковой переменной

In [32]:
#уникальные значения
df['model'].unique()

array(['Сессна 208 Караван', 'Бомбардье CRJ-200', 'Сухой Суперджет-100',
       'Аэробус A319-100', 'Боинг 737-300', 'Аэробус A320-200',
       'Аэробус A321-200', 'Боинг 767-300', 'Боинг 777-300'], dtype=object)

In [33]:
#количество уникальных значемний
len(df['model'].unique())

9

### частота встречающихся значений по строковой переменной

In [36]:
#сравнение условий комфорта относительно модели самолёта, показывает какие условия 
#комфорта на каких моделях самолёта самые популярные.
pd.crosstab(df['fare_conditions'],df['model'])

model,Аэробус A319-100,Аэробус A320-200,Аэробус A321-200,Боинг 737-300,Боинг 767-300,Боинг 777-300,Бомбардье CRJ-200,Сессна 208 Караван,Сухой Суперджет-100
fare_conditions,,,,,,,,,
Business,20,20,28,12,30,30,0,0,12
Comfort,0,0,0,0,0,48,0,0,0
Economy,96,120,142,118,192,324,50,12,85


### минимум, среднее значение, медиана, максимум по числовой переменной

In [38]:
df.describe()

,count_seat,average
count,1339.000000,1339.000000
mean,7841.863331,7841.863331
std,5878.877954,3614.919519
min,622.000000,3552.828358
25%,1695.000000,3552.828358
50%,9529.000000,8748.611765
75%,11626.000000,11160.692308
max,19462.000000,14180.000000
